## $\color{Green}{\text{GMD Table 3: GMSL from Modules}}$

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### $\color{blue}{\text{List of Modules and Data to be used}}$

In [7]:
# Load FACTS workflow data as a dataframe.
df = pd.read_fwf('001_IP_GMD_Table_names_Modules_Data/ModuleTest_DataFile_List_coupling_V2-local.txt',comment = '#') 
#
# Label first column with numbers. (dont to preserve the table structure)
df['Num'] = np.arange(df.shape[0])
# shift column No to first position
first_col = df.pop('Num')
df.insert(0, 'Num', first_col)
df

,Num,dataID,Component,Module,sub_Module,DataFile
0,0,cplng,TE,tlm,sterodynamics,ocean.tlm.sterodynamics_localsl.nc
1,1,ar5k14,Glaciers,kopp14,glaciers,k14glaciers.kopp14.glaciers_localsl.nc
2,2,cplng,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_localsl.nc
3,3,ar5k14,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_localsl.nc
4,4,ar5k14,Glaciers,ipccar6,gmipemuglaciers,XXX
5,5,cplng,Glaciers,emulandice,glaciers,emuglaciers.emulandice.glaciers_localsl.nc
6,6,ar5k14,GrIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_GIS_localsl.nc
7,7,ar5k14,GrIS,ipccar6,ismipemuicesheets,XXX
8,8,ar5k14,GrIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_GIS_localsl.nc
9,9,cplng,GrIS,FittedISMIP,GrIS,GrIS1f.FittedISMIP.GrIS_GIS_localsl.nc


#### $\color{Red}{\text{Load Module data from ==> facts-experiments}}$

In [8]:
EXPDATE     = 221217
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585'];
SCENARIOR   = ['rcp26', 'rcp45', 'rcp85'];
UNIT        = 1000 # (get in meters), divide by 
YEAR        = 2100; # Percentile Year for the table.    
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#       
for val in MODNO:
    #
    for scenario in enumerate(SCENARIO):
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val]; DATAFILE = df["DataFile"][val];
        NUM = df["Num"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        if df["dataID"][val] == 'cplng':
            dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/coupling.{arg1}/output/'.format(arg1=scenario[1],arg2=EXPDATE)
            dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario[1]) + df["DataFile"][val]
        elif df["dataID"][val] == 'ar5k14':
            dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/ar5k14.{arg1}/output/'.format(arg1=SCENARIOR[scenario[0]],arg2=EXPDATE)
            dataFILE    = 'ar5k14.{arg1}.'.format(arg1=SCENARIOR[scenario[0]]) + df["DataFile"][val]
            #
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        if SUB_MODULE == 'thermalexpansion':
            Yind        = np.where(d_nc["years"].values == YEAR)[0][0]
        else:
            Yind        = np.where(d_nc["years"].values == YEAR)[0][0]
        # 
        if SUB_MODULE == 'temperature':
            GMSL    = (d_nc["surface_temperature"][:,Yind,0].values) 
        else:
            GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/UNIT       
        # Find Percentiles.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        MODULE_PAPER = MODULE+'/'+SUB_MODULE
        SCENARIO_PAPER = scenario[1]+'/'+SCENARIOR[scenario[0]]
        TEMP = [NUM,COMP,MODULE_PAPER, SCENARIO_PAPER] + pcntle.tolist()
        df_TEMP.append( TEMP )
df_ptile = pd.DataFrame( df_TEMP, columns=['Num','Component','Module','SSP/RCP', ] + [ f'col_{x}' for xi, x in enumerate( percentList )] )
#
df_ptile[''] = df_ptile.apply(lambda x: f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', axis=1 )
df_ptile1    = pd.DataFrame( df_ptile.set_index( ['Num','Component','Module','SSP/RCP'] )[''] ).unstack().swaplevel( 0,1, axis=1 )
print('Percentile of GMSL [ median(17th,83rd) ] in m for different modules for the year ', YEAR)
df_ptile1

Percentile of GMSL [ median(17th,83rd) ] in m for different modules for the year  2100


,,SSP/RCP,ssp126/rcp26,ssp245/rcp45,ssp585/rcp85
,,,,,
Num,Component,Module,,,
0,TE,tlm/sterodynamics,"0.24 (0.09 , 0.39)","0.34 (0.20 , 0.49)","0.49 (0.34 , 0.65)"
1,Glaciers,kopp14/glaciers,"0.09 (0.07 , 0.11)","0.10 (0.08 , 0.13)","0.13 (0.11 , 0.16)"
2,Glaciers,ipccar5/glaciers,"0.07 (0.05 , 0.10)","0.09 (0.07 , 0.12)","0.13 (0.09 , 0.17)"
3,Glaciers,ipccar5/glaciers,"0.08 (0.05 , 0.10)","0.09 (0.06 , 0.13)","0.13 (0.09 , 0.18)"
5,Glaciers,emulandice/glaciers,"0.08 (0.06 , 0.09)","0.10 (0.08 , 0.12)","0.14 (0.12 , 0.17)"
6,GrIS,ipccar5/icesheets,"0.03 (0.02 , 0.05)","0.04 (0.03 , 0.06)","0.07 (0.05 , 0.10)"
8,GrIS,kopp14/icesheets,"0.03 (0.02 , 0.05)","0.04 (0.02 , 0.06)","0.06 (0.03 , 0.11)"
9,GrIS,FittedISMIP/GrIS,"0.03 (0.03 , 0.04)","0.04 (0.04 , 0.05)","0.06 (0.05 , 0.08)"


In [9]:
print(df_ptile1.to_latex(index=True, escape=False))

\begin{tabular}{llllll}
\toprule
   &     & SSP/RCP &        ssp126/rcp26 &        ssp245/rcp45 &        ssp585/rcp85 \\
   &     & {} \\
Num & Component & Module &                     &                     &                     \\
\midrule
0  & TE & tlm/sterodynamics &  0.24 (0.09 , 0.39) &  0.34 (0.20 , 0.49) &  0.49 (0.34 , 0.65) \\
1  & Glaciers & kopp14/glaciers &  0.09 (0.07 , 0.11) &  0.10 (0.08 , 0.13) &  0.13 (0.11 , 0.16) \\
2  & Glaciers & ipccar5/glaciers &  0.07 (0.05 , 0.10) &  0.09 (0.07 , 0.12) &  0.13 (0.09 , 0.17) \\
3  & Glaciers & ipccar5/glaciers &  0.08 (0.05 , 0.10) &  0.09 (0.06 , 0.13) &  0.13 (0.09 , 0.18) \\
5  & Glaciers & emulandice/glaciers &  0.08 (0.06 , 0.09) &  0.10 (0.08 , 0.12) &  0.14 (0.12 , 0.17) \\
6  & GrIS & ipccar5/icesheets &  0.03 (0.02 , 0.05) &  0.04 (0.03 , 0.06) &  0.07 (0.05 , 0.10) \\
8  & GrIS & kopp14/icesheets &  0.03 (0.02 , 0.05) &  0.04 (0.02 , 0.06) &  0.06 (0.03 , 0.11) \\
9  & GrIS & FittedISMIP/GrIS &  0.03 (0.03 , 0.04) &  0

/tmp/ipykernel_33444/122110213.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_ptile1.to_latex(index=True, escape=False))
